In [2]:
import pandas as pd
from pathlib import Path
import csv
from datetime import datetime, date
import os

In [2]:
p = Path("G:/")

In [3]:
skip_folders = ["_Data_Audits","@Recently-Snapshot","@Recycle","acoustic (WWT-NAS) (Z) - Shortcut.lnk","z_Temp","_Data_Audits",
                "_Data_Sources", "_Fonts", "_Logs", "_Map Layouts", "_Other_data", "_Outputs", "_Projects", "_Scripts", "_Toolboxes", 
                "_Useful"]

In [3]:
Path.cwd()

WindowsPath('c:/Users/Lara/Work/DataDownloads/data-downloads')

Other file types I need to look at (need to decide which raster ones):
- geopackage (unlikely to be many)
- geojson (again unlikely to be many)
- tif/ tiff/ tff


In [7]:
root_dir = Path("G:/")
skip_folders = {"_Data_Audits","@Recently-Snapshot","@Recycle","acoustic (WWT-NAS) (Z) - Shortcut","z_Temp","_Data_Audits",
                "_Data_Sources", "_Fonts", "_Logs", "_Map Layouts", "_Other_data", "_Outputs", "_Projects", "_Scripts", "_Toolboxes", 
                "_Useful", "z_Temp", "Drone_images","Archive" }
output_csv = "shapefiles.csv"


paths_only = []     # list of lists (each = path components only)
dates_only = []     # parallel list of modified dates

max_path_len = 0    # track longest path before adding the date column

# Walk with pruning
for root, dirs, files in os.walk(root_dir):
    dirs[:] = [d for d in dirs if d not in skip_folders]

    for file in files:
        if not file.lower().endswith(".shp"):
            continue

        shp = Path(root) / file

        # Split path into components (WITHOUT date)
        parts = list(shp.parts)
        paths_only.append(parts)

        # Track max parts length
        max_path_len = max(max_path_len, len(parts))

        # Filesystem modified date (store separately)
        modified_ts = shp.stat().st_mtime
        modified_dt = datetime.fromtimestamp(modified_ts).strftime("%Y-%m-%d %H:%M:%S")
        dates_only.append(modified_dt)

# Build header: level_0 ... level_n + date_modified
header = [f"level_{i}" for i in range(max_path_len)] + ["date_modified"]

# Build final rows, padding all path columns BEFORE adding date column
final_rows = []
for parts, modified_dt in zip(paths_only, dates_only):
    padded = parts + [""] * (max_path_len - len(parts))
    padded.append(modified_dt)
    final_rows.append(padded)

# Write CSV
with open(output_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(final_rows)
